## Demo Scenario
We are going to be analyzing some rental data

In [1]:
import pandas as pd
# load data
contacts = pd.read_csv("/Users/yifanwu/Dev/midas/notebooks/rental/contacts.csv")
listings = pd.read_csv("/Users/yifanwu/Dev/midas/notebooks/rental/listings.csv")
users = pd.read_csv("/Users/yifanwu/Dev/midas/notebooks/rental/users.csv")

In [3]:
contacts.head(5)

,id_guest_anon,id_host_anon,id_listing_anon,ts_interaction_first,ts_reply_at_first,ts_accepted_at_first,ts_booking_at,ds_checkin_first,ds_checkout_first,m_guests,m_interactions,m_first_message_length_in_characters,contact_channel_first,guest_user_stage_first
0,da8656a1-51af-4f38-b1c4-94be1f585157,5426897d-960d-4013-9e38-606ae746793c,a408a8b2-0d44-4513-a611-3736d0409bb2,2016-04-21 02:55:53,2016-04-21 03:15:00.0,2016-04-21 03:15:00.0,2016-04-21 03:15:00.0,2016-08-02,2016-08-06,4.0,30,31.0,book_it,past_booker
1,8590d6f1-8bc9-4e8b-bdfb-de78f69fcf37,f30417c5-6df4-45ac-bfc2-6ad1cce398ab,e387c705-0aeb-464c-9375-ece63be6f006,2016-02-16 22:14:01,2016-02-16 23:37:36.0,NaN,NaN,2016-08-11,2016-08-22,5.0,2,312.0,contact_me,past_booker
2,ebcd83ba-bda1-47eb-9680-2dd04ccf3988,13cbf50a-3272-45d4-9866-a06b6ea1b99a,d1eb1960-938f-4305-a353-51e224414dd4,2016-01-27 23:33:38,2016-01-28 02:12:47.0,NaN,NaN,2016-03-14,2016-03-23,1.0,3,164.0,contact_me,new
3,b0af8848-fe2a-4ef1-991e-26ab3066feb3,01614601-d5a4-4776-ab9b-c10d3b865bf0,855f6779-346c-45fc-a64b-ea133250ca7a,2016-05-05 14:42:52,2016-05-05 15:17:40.0,2016-05-05 15:17:40.0,NaN,2016-05-27,2016-05-29,2.0,3,203.0,contact_me,new
4,5ddbbcc3-ac1a-4d8a-b6f8-0f675b76d1b2,f2fed6f3-4c5c-453d-9e64-37c62b8bd06d,f2928a59-c5e7-42b2-9c37-240a40f38dc0,2016-06-23 03:09:25.0,2016-06-23 03:09:26.0,2016-06-23 03:09:33.0,2016-06-23 03:09:33.0,2016-08-19,2016-08-21,3.0,25,0.0,instant_book,past_booker


## Basic Profiling and Visualizations
When loading data into Midas, we show a basic profiler on the side.

In [2]:
from midas import Midas
m = Midas()

ImportError: cannot import name 'RelationalOp' from 'midas.midas_algebra.dataframe' (/Users/yifanwu/Dev/midas/midas/midas_algebra/dataframe.py)

In [ ]:
contacts_df = m.register_df("contacts_df", contacts)

In [ ]:
listings_df = m.register_df("listings_df", listings)
users_df = m.register_df("users_df", users)

In [ ]:
# now let's say that there is an interesting chart the user wants to look at
contact_channel_first_df = contacts_df.project([contacts_df.contact_channel_first]).assign("contact_channel_first_df")


In [ ]:
m_interactions_df = contacts_df.project([contacts_df.m_interactions]).assign("m_interactions_df")


## Access Selections Stream

In [ ]:
contact_channel_stream = contact_channel_first_df.get_stream()
contact_channel_stream

In [ ]:
contact_channel_stream.current

In [ ]:
contacts_df.project([contacts_df.ts_booking_at]).pandas_value

In [ ]:
room_type_df = listings_df.project([listings_df.room_type]).assign("room_type_df")

In [ ]:
# note the code below does not yet work!

In [ ]:
total_reviews_df = listings_df.project([listings_df.total_reviews]).assign("total_reviews_df")


## Add Callbacks to Streams

In [ ]:
def transform(predicate):
    m.ui_comm.send_debug_msg(f"Transform {predicate}")
    new_mdf = contacts_df.apply_selection(contact_channel_stream.current)
    new_mdf.project([new_mdf.m_guests]).assign("m_guests")

contact_channel_stream.add_callback(transform)

In [ ]:
new_mdf = contacts_df.apply_selection(contact_channel_stream.current)
m_guests = new_mdf.project([new_mdf.m_guests]).assign("m_guests")
m_guests.pandas_value

In [ ]:
contact_channel_stream.ref_to_predicate_list

In [ ]:
m.event_loop.tick_funcs['contact_channel_first_df'] = []

In [ ]:
p = contact_channel_stream.current
m.event_loop.tick('contact_channel_first_df', p)

In [ ]:
predicate = contact_channel_stream.current
new_mdf = contacts_df.apply_selection(contact_channel_stream.current)
m_guests = new_mdf.project([new_mdf.m_guests]).assign("m_guests") 


In [ ]:
m_guests.pan

In [ ]:
m_guests_df = new_mdf.project([new_mdf.m_guests]).assign("m_guests")


In [ ]:
p = contacts_df.pandas_value

p[p["contact_channel_first"].isin(['contact_me', 'book_it'])]


In [ ]:
contact_channel_first_df.pandas_value

In [ ]:
new_mdf.pandas_value

In [ ]:
new_mdf.ops.predicate.operation

## Linking Static Charts

## UI Shortcuts